In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [53]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import re  
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/emekaborisama/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/emekaborisama/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/emekaborisama/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emekaborisama/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [174]:
df_train = pd.read_csv('updated_train.csv')
df_test = pd.read_csv('updated_test.csv') #read dataset

In [175]:
df_train.isnull().sum()

ID        0
text      0
target    0
dtype: int64

In [176]:
df_train.head()

,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,266 Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0


In [177]:
df_train['text'] = df_train['text'].astype('str')

In [178]:
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)
    
  return input_txt

In [179]:
df_train['text'] = np.vectorize(remove_pattern)(df_train['text'], "@[\w]*") #remove @ and * from tweet
df_train['text'] = df_train['text'].str.replace("[^a-zA-Z#]", " ") # remove special characters, numbers, punctuations




df_test['text'] = np.vectorize(remove_pattern)(df_test['text'], "@[\w]*") #remove @ and * from tweet
df_test['text'] = df_test['text'].str.replace("[^a-zA-Z#]", " ") 

In [185]:
df_train.head()

,ID,text,target,No_Token,No_Characters
0,train_0,The bitcoin halving is cancelled due to,1,7,39
1,train_1,MercyOfAllah In good times wrapped in its gran...,0,20,117
2,train_2,Days No Digital India No Murder of e learn...,1,19,93
3,train_3,India is likely to run out of the remaining RN...,1,23,112
4,train_4,In these tough times the best way to grow is t...,0,26,121


In [184]:
tokens = []
#def num_token(text):
for line in df_train.text:
  token = nltk.word_tokenize(line)
  tokens.append(len(token))

df_train['No_Token'] = tokens

def split(text):
  return text

char = []
for x in df_train.text.iloc[0:]:
  chara = split(x)
  char.append(len(chara))

df_train['No_Characters'] = char

In [151]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(max_df = 0.9, min_df = 0.1)

In [152]:
from sklearn.feature_extraction.text import CountVectorizer 

In [160]:
cv = CountVectorizer(min_df = 1, max_df = 2)

In [189]:
X = df_train.drop(['ID', 'text', 'target'], axis = 1)
y = df_train.target


In [162]:
x_test = df_test.drop(['ID'], axis = 1)

In [163]:
x_test = cv.fit_transform(df_test['text'])

In [191]:
from sklearn.model_selection import train_test_split #Cross Validation 


In [192]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

## Random forest model

In [193]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=100, random_state=0, n_estimators = 1000)
RF = clf.fit(X_train, y_train)

In [194]:
RF_train = RF.predict(X_train)

print('Train accuracy = {}'.format(
        accuracy_score(RF_train , y_train) * 100)
     )
f1_score = metrics.f1_score(RF_train, y_train, average='macro')
print(' F1 Train classification score: {}'.format(f1_score* 100))

RF_test= RF.predict(X_test)

print('Test accuracy = {}'.format(
        accuracy_score(RF_test, y_test) * 100)
     )
f1_score = metrics.f1_score(RF_test, y_test, average='macro')
print(' F1 Test classification score: {}'.format(f1_score* 100))

Train accuracy = 75.40789784819107
 F1 Train classification score: 75.35657149875455
Test accuracy = 55.765595463138
 F1 Test classification score: 55.76543739279588


In [84]:
RF_test1= RF.predict(X_test).clip(0.1, 0.70)

In [86]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [87]:
rfc=RandomForestClassifier(random_state=42)

In [88]:
param_grid1 = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [91]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid1, cv= 5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                  

In [92]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 200}

In [93]:
rfc1=RandomForestClassifier(random_state=42, 
                            max_features='auto', 
                            n_estimators= 200, max_depth=8, criterion='gini')

In [94]:
rfc1 = clf.fit(X_train, y_train)

In [395]:
pred = rfc1.predict(X_test).clip(0.1, 0.7)

In [95]:
Rfc_train = rfc1.predict(X_train)

print('Train accuracy = {}'.format(
        accuracy_score(Rfc_train , y_train) * 100)
     )
f1_score = metrics.f1_score(Rfc_train, y_train, average='macro')
print(' F1 Train classification score: {}'.format(f1_score* 100))

Rfc_test= rfc1.predict(X_test)

print('Test accuracy = {}'.format(
        accuracy_score(Rfc_test, y_test) * 100)
     )
f1_score = metrics.f1_score(Rfc_test, y_test, average='macro')
print(' F1 Test classification score: {}'.format(f1_score* 100))

Train accuracy = 84.91369117994797
 F1 Train classification score: 84.90473671230657
Test accuracy = 65.21739130434783
 F1 Test classification score: 65.21428367212741


In [128]:
Rfc_test= rfc1.predict(X_test)
print(Rfc_test)

[0 0 0 ... 1 0 0]


In [396]:
log_loss(valid_tag, pred, eps=1e-15)

0.36926563864998846

In [195]:
tokens = []
def num_token(df):
    for line in df:
        token = nltk.word_tokenize(line)
        tokens.append(len(token))
        df['No_Token'] = tokens

def split(text):
    return text

char = []
def char(df):
    for x in df.iloc[0:]:
        chara = split(x)
        char.append(len(chara))
        df['No_Characters'] = char

In [285]:
df = input(str(input))
def data_pro(text):
    df = {'text':[text]}
    df = pd.DataFrame(df)
    return df

#df = data_pro(text)

tokens = []
def num_token(df):
    for line in df.text:
        token = nltk.word_tokenize(line)
        tokens.append(len(token))
        df['No_Token'] = pd.Series(tokens)

#num_token(df)

charater = []
def char(df):
    for x in df.text.iloc[0:]:
        chara = split(x)
        charater.append(len(chara))
        df['No_Characters'] = charater
        
#char(df)

df = data_pro(text)
num_token(df)
char(df)
df = df.drop(['text'], axis = 1)

print(df)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x105636590>> Hello do you know me


   No_Token  No_Characters
0         5             20


In [286]:
RF.predict(df)

array([1])

In [287]:
import pickle

In [295]:
# save the model to disk
filename = 'model.pkl'
pickle.dump(RF, open(filename, 'wb'))

## Gradient Classifer start here

In [665]:
from sklearn import ensemble
clf = ensemble.GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=120,
                                                  n_iter_no_change=None,
                                                  presort='deprecated',
                                                  random_state=None,
                                                  subsample=1.0, tol=0.0001,
                                                  validation_fraction=0.1,
                                                  verbose=0, warm_start=False)
gbc = clf.fit(train_set, train_tag)

In [666]:
gbct= gbc.predict(valid_set).clip(0.1, 0.7)


In [667]:
log_loss(valid_tag, gbct, eps=1e-15)

0.36760287778234535

In [271]:
gbct = gbc.predict(valid_set).clip(0.1111, 0.80)
log_loss(valid_tag, gbct, eps=1e-15)

0.35746470888000365

In [289]:
gbct = gbc.predict(valid_set).clip(0.011, 0.550)
log_loss(valid_tag, gbct, eps=1e-15)

0.4106107466635509

In [163]:
param_grid = { 
    'n_estimators': [200, 1000],
    
}

In [164]:
CV_gbc = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 5)

In [162]:
CV_gbc.fit(train_set, train_tag)

GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

## Gradient end here

## Lightgbm

In [485]:
# build the lightgbm model
import lightgbm as lgb


In [549]:
clf1 = lgb.LGBMClassifier(boosting_type = 'dart', n_estimators = 100, n_iteration = 100,
                         learning_rate=0.1, max_bin = 10, num_leave = 9)
clf1.fit(train_set, train_tag)

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_bin=10,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_iteration=100, n_jobs=-1,
               num_leave=9, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [550]:
clf_t = clf1.predict(valid_set).clip(0.1, 0.8)
log_loss(valid_tag, clf_t, eps=1 e-15)

0.3390220359149719

In [542]:
submission = pd.DataFrame({"ID":df_test["ID"],
                           "target":clf_t})
submission.to_csv("submit7b.csv", index=False)